We would've seen most of these iteration tools in the previous sections and you'll recognise that they're all **lazy iterators** and not **iterables**, making them highly efficient.

# 01 - Aggregators

# 02 - Slicing Iterables

# 03 - Selecting and Filtering

# 04 - Infinite Iterators

# 05 - Chaining and Teeing Iterators

# 06 - Mapping and Reducing

# 07 - Zipping

# 08 - Grouping

# 09 - Combinatorics